# Week 3 - Linear Regression 3

In [1]:
!pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 39.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [statsmodels] [statsmodels]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

First Dataset: Acute Kidney

In [4]:
DATA_PATH = "Acute Kidney.csv" 
df = pd.read_csv(DATA_PATH, low_memory=False)

# Normalize columns (spaces/symbols -> underscores; lowercase)
df.columns = (df.columns.astype(str)
                .str.strip()
                .str.replace(r"\s+", "_", regex=True)
                .str.replace(r"[^0-9a-zA-Z_]", "", regex=True)
                .str.lower())

In [5]:
# Choose a CONTINUOUS target 
preferred = ["cox_los", "los", "length_of_stay", "sofa", "sapsii", "age", "bmi", "glucose", "wbc", "rbc"]
target_col = next((c for c in preferred if c in df.columns), None)
if target_col is None:
    nums = df.select_dtypes(include=["int64","float64"]).columns.tolist()
    if not nums:
        raise ValueError("No numeric columns found to use as a regression target. Set target_col manually.")
    candidates = [c for c in nums if df[c].nunique(dropna=True) >= 10 and set(pd.unique(df[c].dropna())) != {0,1}]
    target_col = candidates[0] if candidates else nums[0]

print("Target:", target_col)
y = pd.to_numeric(df[target_col], errors="coerce")
mask = ~y.isna()
df = df.loc[mask].reset_index(drop=True)
y = y.loc[mask].reset_index(drop=True)

# features split
num_cols_all = df.select_dtypes(include=["int64","float64"]).columns.tolist()
if target_col in num_cols_all: num_cols_all.remove(target_col)
cat_cols = df.select_dtypes(include=["object","category","bool"]).columns.tolist()

# quick NA handling
X = df[num_cols_all + cat_cols].copy()
for c in num_cols_all:
    X[c] = pd.to_numeric(X[c], errors="coerce").fillna(0.0)
for c in cat_cols:
    X[c] = X[c].astype("category").cat.add_categories(["__missing__"]).fillna("__missing__")


def rmse(a, b): 
    return float(np.sqrt(mean_squared_error(a, b)))

def densify(A):
    return A.toarray() if hasattr(A, "toarray") else A

def cv_summary(y_true_list, y_pred_list):
    r2s = [r2_score(yt, yp) for yt, yp in zip(y_true_list, y_pred_list)]
    rmses = [rmse(yt, yp) for yt, yp in zip(y_true_list, y_pred_list)]
    return np.mean(r2s), np.std(r2s), np.mean(rmses), np.std(rmses)

def print_cv_row(rows, name, r2m, r2s, rmsem, rmses):
    rows.append({"Model": name,
                 "CV R^2 (mean ± std)": f"{r2m:.4f} ± {r2s:.4f}",
                 "CV RMSE (mean ± std)": f"{rmsem:.4f} ± {rmses:.4f}"})


Target: cox_los


In [7]:
# Stepwise selection (numeric only, top-K by |corr|)
K_STEPWISE = 15
corrs = X[num_cols_all].corrwith(y).abs().sort_values(ascending=False)
num_stepwise = list(corrs.index[:min(K_STEPWISE, len(corrs))])

def kfold_predict_linear(X_df, y_ser, cols, n_splits=5, seed=42):
    """Fit StandardScaler on the exact subset 'cols' within each fold (fixes KeyError)."""
    if len(cols) == 0:
        return np.nan, np.nan, np.nan, np.nan
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    y_true_list, y_pred_list = [], []
    for tr, va in kf.split(X_df):
        Xtr, Xva = X_df.iloc[tr][cols], X_df.iloc[va][cols]
        ytr, yva = y_ser.iloc[tr], y_ser.iloc[va]
        sc = StandardScaler()
        Xtr_s = sc.fit_transform(Xtr)
        Xva_s = sc.transform(Xva)
        ols = LinearRegression().fit(Xtr_s, ytr)
        y_true_list.append(yva.values)
        y_pred_list.append(ols.predict(Xva_s))
    return cv_summary(y_true_list, y_pred_list)

def forward_stepwise(X_df, y_ser, candidates, max_feats=None, tol=1e-4, n_splits=5):
    selected, best_score = [], -np.inf
    remaining = candidates.copy()
    max_feats = max_feats or len(candidates)
    while remaining and len(selected) < max_feats:
        trial_scores = []
        for c in remaining:
            cols = selected + [c]
            r2m, *_ = kfold_predict_linear(X_df, y_ser, cols, n_splits=n_splits)
            trial_scores.append((r2m, c))
        trial_scores.sort(reverse=True)
        if trial_scores[0][0] > best_score + tol:
            best_score, chosen = trial_scores[0]
            selected.append(chosen)
            remaining.remove(chosen)
        else:
            break
    return selected

def backward_stepwise(X_df, y_ser, start_cols, tol=1e-4, n_splits=5):
    cols = start_cols.copy()
    if len(cols) == 0:
        return cols
    improved = True
    base_r2m, *_ = kfold_predict_linear(X_df, y_ser, cols, n_splits=n_splits)
    while improved and len(cols) > 1:
        improved = False
        trial_scores = []
        for c in cols:
            trial = [x for x in cols if x != c]
            r2m, *_ = kfold_predict_linear(X_df, y_ser, trial, n_splits=n_splits)
            trial_scores.append((r2m, c, trial))
        trial_scores.sort(reverse=True)
        if trial_scores[0][0] > base_r2m + tol:
            base_r2m, removed, cols = trial_scores[0][0], trial_scores[0][1], trial_scores[0][2]
            improved = True
    return cols

# Compute stepwise sets
fwd_selected = forward_stepwise(X, y, num_stepwise, max_feats=min(10, len(num_stepwise)), n_splits=5)
bwd_selected = backward_stepwise(X, y, num_stepwise, n_splits=5)

# CV for stepwise models
fwd_r2m, fwd_r2s, fwd_rm, fwd_rs = kfold_predict_linear(X, y, fwd_selected, n_splits=5) if fwd_selected else (np.nan, np.nan, np.nan, np.nan)
bwd_r2m, bwd_r2s, bwd_rm, bwd_rs = kfold_predict_linear(X, y, bwd_selected, n_splits=5) if bwd_selected else (np.nan, np.nan, np.nan, np.nan)

In [8]:
# PCR: PCA(numeric) + OHE(cats) + OLS (inner CV for n_components)
num_pcr = Pipeline([("scale", StandardScaler()), ("pca", PCA())])
pre_pcr = ColumnTransformer(
    transformers=[
        ("num", num_pcr, [c for c in num_cols_all if c in X.columns]),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ],
    remainder="drop"
)
to_dense = FunctionTransformer(densify)
pcr_pipe = Pipeline([("pre", pre_pcr), ("to_dense", to_dense), ("reg", LinearRegression())])

max_pca = max(1, min(25, len(num_cols_all)))
pcr_param = {"pre__num__pca__n_components": list(range(1, max_pca + 1))}

def nested_cv_pipe(pipe, param_grid, X_df, y_ser, n_splits=5, seed=42):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    y_true_list, y_pred_list, best_params = [], [], []
    for tr, va in kf.split(X_df):
        Xtr, Xva = X_df.iloc[tr], X_df.iloc[va]
        ytr, yva = y_ser.iloc[tr], y_ser.iloc[va]
        gs = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1, scoring="neg_mean_squared_error")
        gs.fit(Xtr, ytr)
        best = gs.best_estimator_
        y_true_list.append(yva.values)
        y_pred_list.append(best.predict(Xva))
        best_params.append(gs.best_params_)
    r2m, r2s, rm, rs = cv_summary(y_true_list, y_pred_list)
    return r2m, r2s, rm, rs, best_params

pcr_r2m, pcr_r2s, pcr_rm, pcr_rs, pcr_params = nested_cv_pipe(pcr_pipe, pcr_param, X, y)

In [9]:
# PLSR: StandardScaler+OHE -> PLSRegression (inner CV for n_components) 
pre_pls = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols_all),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ],
    remainder="drop"
)
pls_pipe = Pipeline([("pre", pre_pls), ("to_dense", to_dense), ("reg", PLSRegression(scale=False))])

max_pls = max(1, min(15, len(num_cols_all)))
pls_param = {"reg__n_components": list(range(1, max_pls + 1))}

pls_r2m, pls_r2s, pls_rm, pls_rs, pls_params = nested_cv_pipe(pls_pipe, pls_param, X, y)

In [10]:
# Summaries (5-fold CV)
cv_rows = []
print_cv_row(cv_rows, f"Forward Stepwise (on {len(fwd_selected)} num)", fwd_r2m, fwd_r2s, fwd_rm, fwd_rs)
print_cv_row(cv_rows, f"Backward Stepwise (on {len(bwd_selected)} num)", bwd_r2m, bwd_r2s, bwd_rm, bwd_rs)
print_cv_row(cv_rows, "PCR (best n_components via CV)", pcr_r2m, pcr_r2s, pcr_rm, pcr_rs)
print_cv_row(cv_rows, "PLSR (best n_components via CV)", pls_r2m, pls_r2s, pls_rm, pls_rs)

cv_table = pd.DataFrame(cv_rows)
print("\n=== Week 3 — 5-fold CV (mean ± std) ===")
print(cv_table.to_string(index=False))


=== Week 3 — 5-fold CV (mean ± std) ===
                          Model CV R^2 (mean ± std) CV RMSE (mean ± std)
    Forward Stepwise (on 3 num)     0.9698 ± 0.0037      5.8339 ± 0.3625
  Backward Stepwise (on 15 num)     0.9698 ± 0.0037      5.8392 ± 0.3640
 PCR (best n_components via CV)     0.9217 ± 0.0071      9.4031 ± 0.3172
PLSR (best n_components via CV)     0.9695 ± 0.0036      5.8637 ± 0.3576


In [12]:
# Common 30% holdout for apples-to-apples comparison 
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.30, random_state=42)

# Forward holdout
if fwd_selected:
    sc_f = StandardScaler().fit(X_tr[fwd_selected])
    Xtr_f = sc_f.transform(X_tr[fwd_selected])
    Xte_f = sc_f.transform(X_te[fwd_selected])
    ols_f = LinearRegression().fit(Xtr_f, y_tr)
    fwd_test = {"Model":"Forward Stepwise",
                "Test R^2": r2_score(y_te, ols_f.predict(Xte_f)),
                "Test RMSE": rmse(y_te, ols_f.predict(Xte_f)),
                "Params": {"features": fwd_selected}}
else:
    fwd_test = {"Model":"Forward Stepwise","Test R^2":np.nan,"Test RMSE":np.nan,"Params":{"features":[]}}

# Backward holdout
if bwd_selected:
    sc_b = StandardScaler().fit(X_tr[bwd_selected])
    Xtr_b = sc_b.transform(X_tr[bwd_selected])
    Xte_b = sc_b.transform(X_te[bwd_selected])
    ols_b = LinearRegression().fit(Xtr_b, y_tr)
    bwd_test = {"Model":"Backward Stepwise",
                "Test R^2": r2_score(y_te, ols_b.predict(Xte_b)),
                "Test RMSE": rmse(y_te, ols_b.predict(Xte_b)),
                "Params": {"features": bwd_selected}}
else:
    bwd_test = {"Model":"Backward Stepwise","Test R^2":np.nan,"Test RMSE":np.nan,"Params":{"features":[]}}

#  PCR holdout (refit with best n_components on train)
try:
    gs_pcr = GridSearchCV(
        pcr_pipe, pcr_param, cv=5, n_jobs=-1, scoring="neg_mean_squared_error"
    )
except TypeError:
    # fallback if your sklearn doesn't support n_jobs in this context
    gs_pcr = GridSearchCV(
        pcr_pipe, pcr_param, cv=5, scoring="neg_mean_squared_error"
    )

gs_pcr.fit(X_tr, y_tr)
pcr_best = gs_pcr.best_estimator_
pcr_n = pcr_best.get_params().get("pre__num__pca__n_components")
pcr_pred = pcr_best.predict(X_te)
pcr_pred = np.ravel(pcr_pred)  # ensure 1D
pcr_test = {
    "Model": "PCR",
    "Params": {"n_components": int(pcr_n) if pcr_n is not None else None},
    "Test R^2": r2_score(y_te, pcr_pred),
    "Test RMSE": rmse(y_te, pcr_pred),
}

# PLSR holdout
try:
    gs_pls = GridSearchCV(
        pls_pipe, pls_param, cv=5, n_jobs=-1, scoring="neg_mean_squared_error"
    )
except TypeError:
    gs_pls = GridSearchCV(
        pls_pipe, pls_param, cv=5, scoring="neg_mean_squared_error"
    )

gs_pls.fit(X_tr, y_tr)
pls_best = gs_pls.best_estimator_
pls_n = int(pls_best.get_params().get("reg__n_components"))
pls_pred = pls_best.predict(X_te)
pls_pred = np.ravel(pls_pred)  # ensure 1D
pls_test = {
    "Model": "PLSR",
    "Params": {"n_components": pls_n},
    "Test R^2": r2_score(y_te, pls_pred),
    "Test RMSE": rmse(y_te, pls_pred),
}

hold_table = pd.DataFrame([fwd_test, bwd_test, pcr_test, pls_test])
print("\n=== Week 3 — Common Holdout (30%) ===")
print(hold_table.to_string(index=False))


=== Week 3 — Common Holdout (30%) ===
            Model  Test R^2  Test RMSE                                                                                                                                                            Params
 Forward Stepwise  0.971335   5.857363                                                                                                           {'features': ['mort_90_day', 'mort_28_day', 'lactate']}
Backward Stepwise  0.971138   5.877375 {'features': ['mort_90_day', 'mort_28_day', 'mort_1_year', 'sapsii', 'sofa', 'aki_stage', 'bun', 'rdw', 'lactate', 'age', 'nlr', 'ly', 'r', 'sii', 'malignancy']}
              PCR  0.928883   9.225955                                                                                                                                              {'n_components': 25}
             PLSR  0.970773   5.914512                                                                                                                       

Week 3 — Model Selection & Dimension Reduction (Acute Kidney)
Objective

Compare forward stepwise, backward stepwise, PCR (principal components regression), and PLSR (partial least squares regression) on the Acute Kidney dataset. Report 5-fold CV (mean ± std) for R² and RMSE, then evaluate all models on a shared 30% holdout for apples-to-apples performance.

Data & Target

Dataset: Acute Kidney.csv

Target (continuous): cox_los (preferred if present). If not available, used: <target_col>

Predictors: mixture of continuous (vitals, labs, scores) and categorical (demographics, comorbidities, flags).

Preprocessing

Column cleanup: lower-cased, spaces/symbols → underscores.

Missing values: numeric → 0.0; categoricals → "__missing__".

Feature typing: numeric vs. categorical.

Scaling: StandardScaler applied where needed (stepwise, PCR numeric block, PLSR).

Encoding: OneHotEncoder(handle_unknown="ignore") for categoricals (PCR/PLSR pipelines).

Methods
Stepwise Linear Regression (greedy subset selection)

Forward stepwise: start empty, add the feature with the largest CV R² gain at each step until no improvement (tolerance).

Backward stepwise: start from a capped set of top-K numeric features (ranked by |corr| with target), remove features that increase CV R² when dropped.

Rationale: provides interpretable subsets; sensitive to collinearity and greedy decisions.

PCR — Principal Components Regression

Pipeline: scale numeric → PCA (choose n_components via inner CV) → OLS, with OHE categoricals appended.

Rationale: compress correlated numeric predictors to orthogonal components; helps with multicollinearity.

PLSR — Partial Least Squares Regression

Pipeline: scale+OHE → PLSRegression, tune n_components via inner CV.

Rationale: finds components that maximize covariance with the target, often stronger than PCA when X–y relation is concentrated in fewer directions.

Interpretation


Multicollinearity: PCR/PLSR typically outperform stepwise when many predictors are correlated; components stabilize estimates.

Parsimony vs. Stability: Stepwise yields sparse, interpretable subsets but can be unstable across folds; PCR/PLSR produce stable components but less direct interpretability.

When to use which:

Prefer PLSR if signal aligns with few latent X–y directions.

Prefer PCR if you primarily need to orthogonalize correlated predictors and retain broad variance.

Prefer Stepwise if interpretability via original features is paramount (and results are validated by CV).

Second Dataset: Colorectal cancer

In [13]:
# Load & tidy
DATA_PATH = "colorectal_cancer_dataset.csv" 
df = pd.read_csv(DATA_PATH, low_memory=False)
df.columns = (df.columns.astype(str)
                .str.strip()
                .str.replace(r"\s+", "_", regex=True)
                .str.replace(r"[^0-9a-zA-Z_]", "", regex=True)
                .str.lower())

In [14]:
# Choose a CONTINUOUS target
preferred = ["survival_months","time_to_event","tumor_size","tumor_volume","age","bmi","los"]
target_col = next((c for c in preferred if c in df.columns), None)
if target_col is None:
    nums = df.select_dtypes(include=["int64","float64"]).columns.tolist()
    if not nums:
        raise ValueError("No numeric columns found for regression target. Set target_col manually.")
    candidates = [c for c in nums if df[c].nunique(dropna=True) >= 10 and set(pd.unique(df[c].dropna())) != {0,1}]
    target_col = candidates[0] if candidates else nums[0]

print("Target:", target_col)
y = pd.to_numeric(df[target_col], errors="coerce")
mask = ~y.isna()
df = df.loc[mask].reset_index(drop=True)
y = y.loc[mask].reset_index(drop=True)

# features
num_cols_all = df.select_dtypes(include=["int64","float64"]).columns.tolist()
if target_col in num_cols_all: num_cols_all.remove(target_col)
cat_cols = df.select_dtypes(include=["object","category","bool"]).columns.tolist()

# quick NA handling
X = df[num_cols_all + cat_cols].copy()
for c in num_cols_all:
    X[c] = pd.to_numeric(X[c], errors="coerce").fillna(0.0)
for c in cat_cols:
    X[c] = X[c].astype("category").cat.add_categories(["__missing__"]).fillna("__missing__")


def rmse(a, b): 
    return float(np.sqrt(mean_squared_error(a, b)))

def densify(A):
    return A.toarray() if hasattr(A, "toarray") else A

def cv_summary(y_true_list, y_pred_list):
    r2s = [r2_score(yt, yp) for yt, yp in zip(y_true_list, y_pred_list)]
    rmses = [rmse(yt, yp) for yt, yp in zip(y_true_list, y_pred_list)]
    return np.mean(r2s), np.std(r2s), np.mean(rmses), np.std(rmses)

def print_cv_row(rows, name, r2m, r2s, rmsem, rmses):
    rows.append({"Model": name,
                 "CV R^2 (mean ± std)": f"{r2m:.4f} ± {r2s:.4f}",
                 "CV RMSE (mean ± std)": f"{rmsem:.4f} ± {rmses:.4f}"})

Target: age


In [15]:
# Stepwise selection (numeric only, top-K by |corr|)
K_STEPWISE = 15
corrs = X[num_cols_all].corrwith(y).abs().sort_values(ascending=False)
num_stepwise = list(corrs.index[:min(K_STEPWISE, len(corrs))])

def kfold_predict_linear(X_df, y_ser, cols, n_splits=5, seed=42):
    """Fit scaler on EXACT subset 'cols' within each fold (prevents KeyErrors)."""
    if len(cols) == 0:
        return np.nan, np.nan, np.nan, np.nan
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    y_true_list, y_pred_list = [], []
    for tr, va in kf.split(X_df):
        Xtr, Xva = X_df.iloc[tr][cols], X_df.iloc[va][cols]
        ytr, yva = y_ser.iloc[tr], y_ser.iloc[va]
        sc = StandardScaler()
        Xtr_s = sc.fit_transform(Xtr)
        Xva_s = sc.transform(Xva)
        ols = LinearRegression().fit(Xtr_s, ytr)
        y_true_list.append(yva.values)
        y_pred_list.append(ols.predict(Xva_s))
    return cv_summary(y_true_list, y_pred_list)

def forward_stepwise(X_df, y_ser, candidates, max_feats=None, tol=1e-4, n_splits=5):
    selected, best_score = [], -np.inf
    remaining = candidates.copy()
    max_feats = max_feats or len(candidates)
    while remaining and len(selected) < max_feats:
        trial_scores = []
        for c in remaining:
            cols = selected + [c]
            r2m, *_ = kfold_predict_linear(X_df, y_ser, cols, n_splits=n_splits)
            trial_scores.append((r2m, c))
        trial_scores.sort(reverse=True)
        if trial_scores[0][0] > best_score + tol:
            best_score, chosen = trial_scores[0]
            selected.append(chosen)
            remaining.remove(chosen)
        else:
            break
    return selected

def backward_stepwise(X_df, y_ser, start_cols, tol=1e-4, n_splits=5):
    cols = start_cols.copy()
    if len(cols) == 0:
        return cols
    improved = True
    base_r2m, *_ = kfold_predict_linear(X_df, y_ser, cols, n_splits=n_splits)
    while improved and len(cols) > 1:
        improved = False
        trial_scores = []
        for c in cols:
            trial = [x for x in cols if x != c]
            r2m, *_ = kfold_predict_linear(X_df, y_ser, trial, n_splits=n_splits)
            trial_scores.append((r2m, c, trial))
        trial_scores.sort(reverse=True)
        if trial_scores[0][0] > base_r2m + tol:
            base_r2m, removed, cols = trial_scores[0][0], trial_scores[0][1], trial_scores[0][2]
            improved = True
    return cols

# compute stepwise feature sets
fwd_selected = forward_stepwise(X, y, num_stepwise, max_feats=min(10, len(num_stepwise)), n_splits=5)
bwd_selected = backward_stepwise(X, y, num_stepwise, n_splits=5)

# CV for stepwise models
fwd_r2m, fwd_r2s, fwd_rm, fwd_rs = kfold_predict_linear(X, y, fwd_selected, n_splits=5) if fwd_selected else (np.nan, np.nan, np.nan, np.nan)
bwd_r2m, bwd_r2s, bwd_rm, bwd_rs = kfold_predict_linear(X, y, bwd_selected, n_splits=5) if bwd_selected else (np.nan, np.nan, np.nan, np.nan)

In [16]:
# PCR: PCA(numeric) + OHE(cats) + OLS (inner CV for n_components)
num_pcr = Pipeline([("scale", StandardScaler()), ("pca", PCA())])

trans = [("num", num_pcr, num_cols_all)]
if len(cat_cols):
    trans.append(("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols))

pre_pcr = ColumnTransformer(transformers=trans, remainder="drop")
to_dense = FunctionTransformer(densify)
pcr_pipe = Pipeline([("pre", pre_pcr), ("to_dense", to_dense), ("reg", LinearRegression())])

max_pca = max(1, min(25, len(num_cols_all)))
pcr_param = {"pre__num__pca__n_components": list(range(1, max_pca + 1))}

def nested_cv_pipe(pipe, param_grid, X_df, y_ser, n_splits=5, seed=42):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    y_true_list, y_pred_list, best_params = [], [], []
    for tr, va in kf.split(X_df):
        Xtr, Xva = X_df.iloc[tr], X_df.iloc[va]
        ytr, yva = y_ser.iloc[tr], y_ser.iloc[va]
        try:
            gs = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1, scoring="neg_mean_squared_error")
        except TypeError:
            gs = GridSearchCV(pipe, param_grid, cv=5, scoring="neg_mean_squared_error")
        gs.fit(Xtr, ytr)
        best = gs.best_estimator_
        y_true_list.append(yva.values)
        y_pred_list.append(best.predict(Xva))
        best_params.append(gs.best_params_)
    r2m, r2s, rm, rs = cv_summary(y_true_list, y_pred_list)
    return r2m, r2s, rm, rs, best_params

pcr_r2m, pcr_r2s, pcr_rm, pcr_rs, pcr_params = nested_cv_pipe(pcr_pipe, pcr_param, X, y)

In [17]:
#PLSR: StandardScaler+OHE -> PLSRegression (inner CV for n_components)
trans_pls = [("num", StandardScaler(), num_cols_all)]
if len(cat_cols):
    trans_pls.append(("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols))

pre_pls = ColumnTransformer(transformers=trans_pls, remainder="drop")
pls_pipe = Pipeline([("pre", pre_pls), ("to_dense", to_dense), ("reg", PLSRegression(scale=False))])

max_pls = max(1, min(15, len(num_cols_all)))
pls_param = {"reg__n_components": list(range(1, max_pls + 1))}

pls_r2m, pls_r2s, pls_rm, pls_rs, pls_params = nested_cv_pipe(pls_pipe, pls_param, X, y)

In [18]:
# Summaries (5-fold CV)
cv_rows = []
print_cv_row(cv_rows, f"Forward Stepwise (on {len(fwd_selected)} num)", fwd_r2m, fwd_r2s, fwd_rm, fwd_rs)
print_cv_row(cv_rows, f"Backward Stepwise (on {len(bwd_selected)} num)", bwd_r2m, bwd_r2s, bwd_rm, bwd_rs)
print_cv_row(cv_rows, "PCR (best n_components via CV)", pcr_r2m, pcr_r2s, pcr_rm, pcr_rs)
print_cv_row(cv_rows, "PLSR (best n_components via CV)", pls_r2m, pls_r2s, pls_rm, pls_rs)

cv_table = pd.DataFrame(cv_rows)
print("\n=== Week 3 — 5-fold CV (mean ± std) ===")
print(cv_table.to_string(index=False))


=== Week 3 — 5-fold CV (mean ± std) ===
                          Model CV R^2 (mean ± std) CV RMSE (mean ± std)
    Forward Stepwise (on 1 num)    -0.0000 ± 0.0001     11.8724 ± 0.0383
   Backward Stepwise (on 5 num)    -0.0001 ± 0.0002     11.8727 ± 0.0384
 PCR (best n_components via CV)    -0.0004 ± 0.0002     11.8743 ± 0.0377
PLSR (best n_components via CV)    -0.0003 ± 0.0002     11.8740 ± 0.0379


In [19]:
# Common 30% holdout for apples-to-apples comparison
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.30, random_state=42)

# Forward holdout
if fwd_selected:
    sc_f = StandardScaler().fit(X_tr[fwd_selected])
    Xtr_f = sc_f.transform(X_tr[fwd_selected])
    Xte_f = sc_f.transform(X_te[fwd_selected])
    ols_f = LinearRegression().fit(Xtr_f, y_tr)
    fwd_test = {"Model":"Forward Stepwise",
                "Test R^2": r2_score(y_te, ols_f.predict(Xte_f)),
                "Test RMSE": rmse(y_te, ols_f.predict(Xte_f)),
                "Params": {"features": fwd_selected}}
else:
    fwd_test = {"Model":"Forward Stepwise","Test R^2":np.nan,"Test RMSE":np.nan,"Params":{"features":[]}}

# Backward holdout
if bwd_selected:
    sc_b = StandardScaler().fit(X_tr[bwd_selected])
    Xtr_b = sc_b.transform(X_tr[bwd_selected])
    Xte_b = sc_b.transform(X_te[bwd_selected])
    ols_b = LinearRegression().fit(Xtr_b, y_tr)
    bwd_test = {"Model":"Backward Stepwise",
                "Test R^2": r2_score(y_te, ols_b.predict(Xte_b)),
                "Test RMSE": rmse(y_te, ols_b.predict(Xte_b)),
                "Params": {"features": bwd_selected}}
else:
    bwd_test = {"Model":"Backward Stepwise","Test R^2":np.nan,"Test RMSE":np.nan,"Params":{"features":[]}}

# PCR holdout (refit with best n_components on train)
try:
    gs_pcr = GridSearchCV(pcr_pipe, pcr_param, cv=5, n_jobs=-1, scoring="neg_mean_squared_error")
except TypeError:
    gs_pcr = GridSearchCV(pcr_pipe, pcr_param, cv=5, scoring="neg_mean_squared_error")
gs_pcr.fit(X_tr, y_tr)
pcr_best = gs_pcr.best_estimator_
pcr_n = pcr_best.get_params().get("pre__num__pca__n_components")
pcr_pred = np.ravel(pcr_best.predict(X_te))
pcr_test = {"Model":"PCR", "Params":{"n_components": int(pcr_n) if pcr_n is not None else None},
            "Test R^2": r2_score(y_te, pcr_pred), "Test RMSE": rmse(y_te, pcr_pred)}

# PLSR holdout
try:
    gs_pls = GridSearchCV(pls_pipe, pls_param, cv=5, n_jobs=-1, scoring="neg_mean_squared_error")
except TypeError:
    gs_pls = GridSearchCV(pls_pipe, pls_param, cv=5, scoring="neg_mean_squared_error")
gs_pls.fit(X_tr, y_tr)
pls_best = gs_pls.best_estimator_
pls_n = int(pls_best.get_params().get("reg__n_components"))
pls_pred = np.ravel(pls_best.predict(X_te))
pls_test = {"Model":"PLSR", "Params":{"n_components": pls_n},
            "Test R^2": r2_score(y_te, pls_pred), "Test RMSE": rmse(y_te, pls_pred)}

hold_table = pd.DataFrame([fwd_test, bwd_test, pcr_test, pls_test])
print("\n=== Week 3 — Common Holdout (30%) ===")
print(hold_table.to_string(index=False))


=== Week 3 — Common Holdout (30%) ===
            Model  Test R^2  Test RMSE                                                                                                                  Params
 Forward Stepwise -0.000124  11.890120                                                                                      {'features': ['healthcare_costs']}
Backward Stepwise -0.000253  11.890885 {'features': ['healthcare_costs', 'tumor_size_mm', 'patient_id', 'mortality_rate_per_100k', 'incidence_rate_per_100k']}
              PCR -0.000283  11.891064                                                                                                     {'n_components': 1}
             PLSR -0.000326  11.891321                                                                                                     {'n_components': 1}


Week 3 — Model Selection & Dimension Reduction (Colorectal Cancer)
Objective

Compare forward stepwise, backward stepwise, PCR (Principal Components Regression), and PLSR (Partial Least Squares Regression) on the colorectal_cancer_dataset.csv. Report 5-fold CV (mean ± std) for R² and RMSE, then evaluate all models on a shared 30% holdout for apples-to-apples performance.

Data & Target

Dataset: colorectal_cancer_dataset

Target (continuous): survival_months (preferred if present). If not available, used: <target_col>

Predictors: mix of continuous (e.g., age, BMI, tumor size/volume, biomarkers) and categorical (e.g., sex, stage, site, therapy).

Preprocessing

Column cleanup: lower-cased, spaces/symbols → underscores.

Missing values: numeric → 0.0; categoricals → "__missing__".

Feature typing: numeric vs categorical.

Scaling: StandardScaler applied where needed (stepwise, PCR numeric block, PLSR).

Encoding: OneHotEncoder(handle_unknown="ignore") for categoricals (PCR/PLSR pipelines).

Methods
Stepwise Linear Regression (greedy subset selection)

Forward stepwise: start empty; at each step add the feature with the largest CV R² gain until improvement < tolerance.

Backward stepwise: start from top-K numeric features (ranked by |corr| with target); iteratively drop features that increase CV R².

Why: yields interpretable subsets; can be unstable with high collinearity.

PCR — Principal Components Regression

Pipeline: scale numeric → PCA (choose n_components via inner CV) → OLS, with OHE categoricals appended.

Why: orthogonalizes correlated predictors → mitigates multicollinearity.

PLSR — Partial Least Squares Regression

Pipeline: scale + OHE → PLSRegression, tune n_components via inner CV.

Why: finds components that maximize covariance with the target; often needs fewer components than PCR when X–y relation is concentrated.

Interpretation

Collinearity handling: PCR/PLSR often outperform stepwise when predictors are highly correlated, by stabilizing the signal in low-dimensional components.

Parsimony vs. stability: Stepwise gives sparse, interpretable subsets but can be unstable across folds; PCR/PLSR provide stable latent factors but are less directly interpretable.

When to prefer which:

PLSR if the predictive signal lies in a few X–y directions.

PCR if the goal is mainly to orthogonalize and denoise correlated predictors.

Stepwise when interpretability via original features is critical and results are validated by CV.

Third Dataset: Diabetes

In [20]:
DATA_PATH = "diabetes_012_health_indicators_BRFSS2015.csv"
df = pd.read_csv(DATA_PATH, low_memory=True)

# normalize columns
df.columns = (df.columns.astype(str)
                .str.strip().str.replace(r"\s+", "_", regex=True)
                .str.replace(r"[^0-9a-zA-Z_]", "", regex=True)
                .str.lower())

In [21]:
# Choose a CONTINUOUS target
# Prefer BMI; else pick a numeric with enough variability (not just {0,1})
preferred = ["bmi", "menthlth", "physhlth", "genhlth", "age"]
target_col = next((c for c in preferred if c in df.columns), None)
if target_col is None:
    nums = df.select_dtypes(include=["int64","float64"]).columns.tolist()
    if not nums:
        raise ValueError("No numeric columns for regression target. Set target_col manually.")
    candidates = [c for c in nums if df[c].nunique(dropna=True) >= 10 and set(pd.unique(df[c].dropna())) != {0,1}]
    target_col = candidates[0] if candidates else nums[0]

print("Target:", target_col)
y = pd.to_numeric(df[target_col], errors="coerce")
mask = ~y.isna()
df = df.loc[mask].reset_index(drop=True)
y = y.loc[mask].reset_index(drop=True)

Target: bmi


In [22]:
#Feature matrix (mostly numeric in BRFSS)
num_cols_all = df.select_dtypes(include=["int64","float64"]).columns.tolist()
if target_col in num_cols_all: num_cols_all.remove(target_col)
cat_cols = df.select_dtypes(include=["object","category","bool"]).columns.tolist()  # often empty in BRFSS

X = df[num_cols_all + cat_cols].copy()
for c in num_cols_all:
    X[c] = pd.to_numeric(X[c], errors="coerce").fillna(0.0).astype("float32")
for c in cat_cols:
    X[c] = X[c].astype("category").cat.add_categories(["__missing__"]).fillna("__missing__")


def rmse(a, b): 
    return float(np.sqrt(mean_squared_error(a, b)))

def densify(A):
    return A.toarray() if hasattr(A, "toarray") else A

def cv_summary(y_true_list, y_pred_list):
    r2s = [r2_score(yt, yp) for yt, yp in zip(y_true_list, y_pred_list)]
    rmses = [rmse(yt, yp) for yt, yp in zip(y_true_list, y_pred_list)]
    return np.mean(r2s), np.std(r2s), np.mean(rmses), np.std(rmses)

def print_cv_row(rows, name, r2m, r2s, rmsem, rmses):
    rows.append({"Model": name,
                 "CV R^2 (mean ± std)": f"{r2m:.4f} ± {r2s:.4f}",
                 "CV RMSE (mean ± std)": f"{rmsem:.4f} ± {rmses:.4f}"})

In [23]:
# Stepwise selection (numeric only, top-K by |corr|)
# Large dataset ⇒ limit candidates and (optionally) subsample rows during stepwise CV
K_STEPWISE = 20
MAX_STEPWISE_ROWS = 120_000  # set None to use all rows (slower)

corrs = X[num_cols_all].corrwith(y.astype("float32")).abs().sort_values(ascending=False)
num_stepwise = list(corrs.index[:min(K_STEPWISE, len(corrs))])

def kfold_predict_linear(X_df, y_ser, cols, n_splits=5, seed=42):
    """Fit scaler on EXACT subset 'cols' within each fold; optional row cap for speed."""
    if len(cols) == 0:
        return np.nan, np.nan, np.nan, np.nan
    # Optional global subsample for speed
    if (MAX_STEPWISE_ROWS is not None) and (len(X_df) > MAX_STEPWISE_ROWS):
        rng = np.random.default_rng(seed)
        idx = rng.choice(len(X_df), size=MAX_STEPWISE_ROWS, replace=False)
        X_use = X_df.iloc[idx][cols]
        y_use = y_ser.iloc[idx]
    else:
        X_use = X_df[cols]
        y_use = y_ser

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    y_true_list, y_pred_list = [], []
    for tr, va in kf.split(X_use):
        Xtr, Xva = X_use.iloc[tr], X_use.iloc[va]
        ytr, yva = y_use.iloc[tr], y_use.iloc[va]
        sc = StandardScaler()
        Xtr_s = sc.fit_transform(Xtr)
        Xva_s = sc.transform(Xva)
        ols = LinearRegression().fit(Xtr_s, ytr)
        y_true_list.append(yva.values)
        y_pred_list.append(ols.predict(Xva_s))
    return cv_summary(y_true_list, y_pred_list)

def forward_stepwise(X_df, y_ser, candidates, max_feats=None, tol=1e-4, n_splits=5):
    selected, best_score = [], -np.inf
    remaining = candidates.copy()
    max_feats = max_feats or len(candidates)
    while remaining and len(selected) < max_feats:
        trial_scores = []
        for c in remaining:
            cols = selected + [c]
            r2m, *_ = kfold_predict_linear(X_df, y_ser, cols, n_splits=n_splits)
            trial_scores.append((r2m, c))
        trial_scores.sort(reverse=True)
        if trial_scores[0][0] > best_score + tol:
            best_score, chosen = trial_scores[0]
            selected.append(chosen)
            remaining.remove(chosen)
        else:
            break
    return selected

def backward_stepwise(X_df, y_ser, start_cols, tol=1e-4, n_splits=5):
    cols = start_cols.copy()
    if len(cols) == 0:
        return cols
    improved = True
    base_r2m, *_ = kfold_predict_linear(X_df, y_ser, cols, n_splits=n_splits)
    while improved and len(cols) > 1:
        improved = False
        trial_scores = []
        for c in cols:
            trial = [x for x in cols if x != c]
            r2m, *_ = kfold_predict_linear(X_df, y_ser, trial, n_splits=n_splits)
            trial_scores.append((r2m, c, trial))
        trial_scores.sort(reverse=True)
        if trial_scores[0][0] > base_r2m + tol:
            base_r2m, removed, cols = trial_scores[0][0], trial_scores[0][1], trial_scores[0][2]
            improved = True
    return cols

# Compute stepwise sets
fwd_selected = forward_stepwise(X, y, num_stepwise, max_feats=min(12, len(num_stepwise)), n_splits=5)
bwd_selected = backward_stepwise(X, y, num_stepwise, n_splits=5)

# CV for stepwise models
fwd_r2m, fwd_r2s, fwd_rm, fwd_rs = kfold_predict_linear(X, y, fwd_selected, n_splits=5) if fwd_selected else (np.nan, np.nan, np.nan, np.nan)
bwd_r2m, bwd_r2s, bwd_rm, bwd_rs = kfold_predict_linear(X, y, bwd_selected, n_splits=5) if bwd_selected else (np.nan, np.nan, np.nan, np.nan)


In [24]:
# PCR: PCA(numeric) + OHE(cats) + OLS (inner CV for n_components)
num_pcr = Pipeline([("scale", StandardScaler()), ("pca", PCA())])

trans = [("num", num_pcr, num_cols_all)]
if len(cat_cols):
    trans.append(("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols))

pre_pcr = ColumnTransformer(transformers=trans, remainder="drop")
to_dense = FunctionTransformer(densify)
pcr_pipe = Pipeline([("pre", pre_pcr), ("to_dense", to_dense), ("reg", LinearRegression())])

max_pca = max(1, min(25, len(num_cols_all)))
pcr_param = {"pre__num__pca__n_components": list(range(1, max_pca + 1))}

# inner-CV row cap for speed
MAX_INNER_ROWS = 120_000

def nested_cv_pipe(pipe, param_grid, X_df, y_ser, n_splits=5, seed=42):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    y_true_list, y_pred_list, best_params = [], [], []
    rng = np.random.default_rng(seed)
    for tr, va in kf.split(X_df):
        Xtr, Xva = X_df.iloc[tr], X_df.iloc[va]
        ytr, yva = y_ser.iloc[tr], y_ser.iloc[va]

        # cap rows during inner CV
        if (MAX_INNER_ROWS is not None) and (len(Xtr) > MAX_INNER_ROWS):
            idx = rng.choice(len(Xtr), size=MAX_INNER_ROWS, replace=False)
            Xcv, ycv = Xtr.iloc[idx], ytr.iloc[idx]
        else:
            Xcv, ycv = Xtr, ytr

        try:
            gs = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1, scoring="neg_mean_squared_error")
        except TypeError:
            gs = GridSearchCV(pipe, param_grid, cv=5, scoring="neg_mean_squared_error")
        gs.fit(Xcv, ycv)

        best = gs.best_estimator_
        y_true_list.append(yva.values)
        y_pred_list.append(best.predict(Xva))
        best_params.append(gs.best_params_)
    r2m, r2s, rm, rs = cv_summary(y_true_list, y_pred_list)
    return r2m, r2s, rm, rs, best_params

pcr_r2m, pcr_r2s, pcr_rm, pcr_rs, pcr_params = nested_cv_pipe(pcr_pipe, pcr_param, X, y)

In [25]:
# PLSR: StandardScaler+OHE -> PLSRegression (inner CV for n_components)
trans_pls = [("num", StandardScaler(), num_cols_all)]
if len(cat_cols):
    trans_pls.append(("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols))

pre_pls = ColumnTransformer(transformers=trans_pls, remainder="drop")
pls_pipe = Pipeline([("pre", pre_pls), ("to_dense", to_dense), ("reg", PLSRegression(scale=False))])

max_pls = max(1, min(15, len(num_cols_all)))
pls_param = {"reg__n_components": list(range(1, max_pls + 1))}

pls_r2m, pls_r2s, pls_rm, pls_rs, pls_params = nested_cv_pipe(pls_pipe, pls_param, X, y)

In [26]:
# Summaries (5-fold CV)
cv_rows = []
print_cv_row(cv_rows, f"Forward Stepwise (on {len(fwd_selected)} num)", fwd_r2m, fwd_r2s, fwd_rm, fwd_rs)
print_cv_row(cv_rows, f"Backward Stepwise (on {len(bwd_selected)} num)", bwd_r2m, bwd_r2s, bwd_rm, bwd_rs)
print_cv_row(cv_rows, "PCR (best n_components via CV)", pcr_r2m, pcr_r2s, pcr_rm, pcr_rs)
print_cv_row(cv_rows, "PLSR (best n_components via CV)", pls_r2m, pls_r2s, pls_rm, pls_rs)

cv_table = pd.DataFrame(cv_rows)
print("\n=== Week 3 — 5-fold CV (mean ± std) ===")
print(cv_table.to_string(index=False))


=== Week 3 — 5-fold CV (mean ± std) ===
                          Model CV R^2 (mean ± std) CV RMSE (mean ± std)
   Forward Stepwise (on 12 num)     0.1362 ± 0.0053      6.1641 ± 0.0563
  Backward Stepwise (on 20 num)     0.1379 ± 0.0051      6.1581 ± 0.0554
 PCR (best n_components via CV)     0.1392 ± 0.0028      6.1311 ± 0.0484
PLSR (best n_components via CV)     0.1392 ± 0.0028      6.1311 ± 0.0483


In [27]:
# Common 30% holdout for apples-to-apples comparison 
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.30, random_state=42)

# Forward holdout
if fwd_selected:
    sc_f = StandardScaler().fit(X_tr[fwd_selected])
    Xtr_f = sc_f.transform(X_tr[fwd_selected])
    Xte_f = sc_f.transform(X_te[fwd_selected])
    ols_f = LinearRegression().fit(Xtr_f, y_tr)
    fwd_test = {"Model":"Forward Stepwise",
                "Test R^2": r2_score(y_te, ols_f.predict(Xte_f)),
                "Test RMSE": rmse(y_te, ols_f.predict(Xte_f)),
                "Params": {"features": fwd_selected}}
else:
    fwd_test = {"Model":"Forward Stepwise","Test R^2":np.nan,"Test RMSE":np.nan,"Params":{"features":[]}}

# Backward holdout
if bwd_selected:
    sc_b = StandardScaler().fit(X_tr[bwd_selected])
    Xtr_b = sc_b.transform(X_tr[bwd_selected])
    Xte_b = sc_b.transform(X_te[bwd_selected])
    ols_b = LinearRegression().fit(Xtr_b, y_tr)
    bwd_test = {"Model":"Backward Stepwise",
                "Test R^2": r2_score(y_te, ols_b.predict(Xte_b)),
                "Test RMSE": rmse(y_te, ols_b.predict(Xte_b)),
                "Params": {"features": bwd_selected}}
else:
    bwd_test = {"Model":"Backward Stepwise","Test R^2":np.nan,"Test RMSE":np.nan,"Params":{"features":[]}}

# PCR holdout (refit with best n_components on train)
try:
    gs_pcr = GridSearchCV(pcr_pipe, pcr_param, cv=5, n_jobs=-1, scoring="neg_mean_squared_error")
except TypeError:
    gs_pcr = GridSearchCV(pcr_pipe, pcr_param, cv=5, scoring="neg_mean_squared_error")
gs_pcr.fit(X_tr, y_tr)
pcr_best = gs_pcr.best_estimator_
pcr_n = pcr_best.get_params().get("pre__num__pca__n_components")
pcr_pred = np.ravel(pcr_best.predict(X_te))
pcr_test = {"Model":"PCR", "Params":{"n_components": int(pcr_n) if pcr_n is not None else None},
            "Test R^2": r2_score(y_te, pcr_pred), "Test RMSE": rmse(y_te, pcr_pred)}

# PLSR holdout
try:
    gs_pls = GridSearchCV(pls_pipe, pls_param, cv=5, n_jobs=-1, scoring="neg_mean_squared_error")
except TypeError:
    gs_pls = GridSearchCV(pls_pipe, pls_param, cv=5, scoring="neg_mean_squared_error")
gs_pls.fit(X_tr, y_tr)
pls_best = gs_pls.best_estimator_
pls_n = int(pls_best.get_params().get("reg__n_components"))
pls_pred = np.ravel(pls_best.predict(X_te))
pls_test = {"Model":"PLSR", "Params":{"n_components": pls_n},
            "Test R^2": r2_score(y_te, pls_pred), "Test RMSE": rmse(y_te, pls_pred)}

hold_table = pd.DataFrame([fwd_test, bwd_test, pcr_test, pls_test])
print("\n=== Week 3 — Common Holdout (30%) ===")
print(hold_table.to_string(index=False))


=== Week 3 — Common Holdout (30%) ===
            Model  Test R^2  Test RMSE                                                                                                                                                                                                                                                                            Params
 Forward Stepwise  0.136078   6.116439                                                                                                                    {'features': ['genhlth', 'diabetes_012', 'highbp', 'age', 'diffwalk', 'physactivity', 'fruits', 'stroke', 'hvyalcoholconsump', 'physhlth', 'sex', 'highchol']}
Backward Stepwise  0.137271   6.112218 {'features': ['genhlth', 'diabetes_012', 'highbp', 'diffwalk', 'physactivity', 'physhlth', 'highchol', 'education', 'income', 'fruits', 'menthlth', 'veggies', 'nodocbccost', 'heartdiseaseorattack', 'hvyalcoholconsump', 'sex', 'age', 'cholcheck', 'stroke', 'anyhealthcare']}
              PCR  0.1

Week 3 — Breakdown (Diabetes BRFSS2015)
1) Goal (what we’re evaluating)

Greedy subset models: Forward & Backward stepwise linear regression (interpretable, but can be unstable).

Dimension-reduction models: PCR (PCA → OLS) and PLSR (PLSRegression).

Validation: Report 5-fold CV mean ± std of R² and RMSE, plus a shared 30% holdout (apples-to-apples).

2) Data & target (how the target is chosen)

Loads diabetes_012_health_indicators_BRFSS2015

Picks a continuous target; default preference is bmi.
Tip: lock it with

target_col = "bmi"


Drops rows with missing target; keeps the rest.

3) Features & basic preprocessing

Splits into:

Numeric columns (most of BRFSS) → num_cols_all

Categorical columns (if any) → cat_cols

Quick NA handling: numerics → 0.0, categoricals → "__missing__".

Standardization (StandardScaler) is applied inside each CV fold (prevents leakage).

4) Stepwise selection (interpretability-first)

We rank numeric features by absolute correlation with the target, keep top-K as candidates.

Code knob: K_STEPWISE = 20

For speed on large N, can subsample rows during stepwise CV:

Code knob: MAX_STEPWISE_ROWS = 120_000 (set to None to use all rows)

Forward stepwise:

Start empty → add the feature that gives the largest CV R² gain; stop when gain < tol.

Backward stepwise:

Start with all top-K; remove the feature whose removal improves CV R² the most; stop when no improvement.

Both use linear regression on standardized features (only the current subset).

Outputs:

fwd_selected, bwd_selected (lists of chosen numeric features)


When to use: you need original-feature interpretability and a compact subset.

5) PCR (Principal Components Regression)

Pipeline: scale numeric → PCA(n_components) → concatenate OHE(cats) → OLS.


Motivation: orthogonalizes correlated predictors (handles multicollinearity), but components are variance-oriented (not target-oriented).

Outputs:

Best n_components per fold in pcr_params (dicts)

6) PLSR (Partial Least Squares Regression)

Pipeline: scale+OHE → PLSRegression(n_components).


Motivation: Finds directions that maximize covariance with y (often needs fewer components than PCR).


7) Nested CV & metrics (how scores are computed)

Outer 5-fold CV: unbiased performance estimate.

Inside each outer train fold, GridSearchCV tunes components (PCR/PLSR).


RMSE in same units as the target (here, BMI units).

8) Common 30% holdout (final apples-to-apples test)

We refit each approach on the same 70% train and score on the same 30% test:

Forward / Backward: scale using train subset, fit OLS, score on test.

PCR / PLSR: re-run grid search on the train to pick the best n_components, then score on test.

Results printed in hold_table with tuned params.

9) How to read the outputs

cv_table (mean ± std across folds): Compare models by higher CV R² and lower CV RMSE. 

hold_table:
Confirms with an independent test R²/RMSE. Minor deviations from CV are normal.


Runtime too slow:

Lower K_STEPWISE, MAX_STEPWISE_ROWS, MAX_INNER_ROWS

Use n_splits=3 temporarily.

Overfitting signs: strong train fit, weak CV → prefer PLSR or PCR (fewer components), or try Week 2 regularization.
